# Data Exploration: BluePrint and Reddit Moderator Perceptions

This notebook explores the two datasets used for simulating social media moderation policies:
1. BluePrint: Human-AI social interactions dataset
2. Reddit Moderator Perceptions: Moderator interpretations of toxic content

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. BluePrint Dataset

In [ ]:
# Load BluePrint dataset from HuggingFace
# Available configurations: "2_clusters", "100_clusters", "1000_clusters"
# Start with 100_clusters as a medium-sized dataset

blueprint = load_dataset("ComplexDataLab/BluePrint", "100_clusters")
print("Dataset splits:", blueprint.keys())
print("\nDataset info:")
print(blueprint)

In [ ]:
print(blueprint['full'].features)

In [ ]:
# Convert to pandas DataFrame
df_blueprint = pd.DataFrame(blueprint['full'])
print(f"Shape: {df_blueprint.shape}")
print(f"\nColumns: {df_blueprint.columns.tolist()}")
print(f"\nFirst few rows:")
df_blueprint.head()

In [ ]:
# Basic statistics
print("Dataset Statistics:")
print(f"Total samples: {len(df_blueprint):,}")
print(f"\nColumn info:")
print(df_blueprint.info())
print(f"\nMissing values:")
print(df_blueprint.isnull().sum())

# Check cluster distribution
print(f"\nNumber of unique clusters: {df_blueprint['cluster_id'].nunique()}")
print(f"\nCluster distribution (top 10):")
print(df_blueprint['cluster_id'].value_counts().head(10))

In [ ]:
# Explore thread structure
print("Thread Structure Analysis:")
print(f"\nSample thread (first row):")
sample_thread = df_blueprint['thread'].iloc[0]
print(type(sample_thread))
print(f"\nThread length: {len(sample_thread) if isinstance(sample_thread, (list, dict)) else 'N/A'}")
print(f"\nThread content preview:")
print(sample_thread if not isinstance(sample_thread, list) or len(sample_thread) < 3 else sample_thread[:3])

In [ ]:
# Analyze thread lengths
thread_lengths = df_blueprint['thread'].apply(lambda x: len(x) if isinstance(x, list) else 1)
print("\nThread Length Statistics:")
print(thread_lengths.describe())

# Visualize cluster distribution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
cluster_counts = df_blueprint['cluster_id'].value_counts()
plt.hist(cluster_counts, bins=30, edgecolor='black', alpha=0.7)
plt.xlabel('Number of Threads per Cluster')
plt.ylabel('Frequency')
plt.title('Distribution of Threads Across Clusters')
plt.yscale('log')

plt.subplot(1, 2, 2)
plt.hist(thread_lengths, bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('Thread Length (number of messages)')
plt.ylabel('Frequency')
plt.title('Distribution of Thread Lengths')
plt.yscale('log')

plt.tight_layout()
plt.show()

In [ ]:
# Deep dive into thread content structure
print("Detailed Thread Content Analysis:")
print("\nExamining first thread in detail:")
first_thread = df_blueprint['thread'].iloc[0]

if isinstance(first_thread, list) and len(first_thread) > 0:
    print(f"\nNumber of messages in thread: {len(first_thread)}")
    print(f"\nFirst message structure:")
    print(type(first_thread[0]))
    print(first_thread[0])
    
    # Check if messages have fields we need for moderation
    if isinstance(first_thread[0], dict):
        print(f"\nMessage fields: {first_thread[0].keys()}")
else:
    print(f"Thread type: {type(first_thread)}")
    print(f"Thread content: {first_thread}")

In [ ]:
# Extract key statistics about message content
print("Message-Level Statistics:")

# Function to safely extract message fields
def extract_message_info(thread):
    if not isinstance(thread, list):
        return None
    
    info = {
        'num_messages': len(thread),
        'unique_users': set(),
        'has_text': False,
        'has_actions': False
    }
    
    for msg in thread:
        if isinstance(msg, dict):
            if 'user_id' in msg:
                info['unique_users'].add(msg['user_id'])
            if 'text' in msg or 'content' in msg:
                info['has_text'] = True
            if 'action' in msg or 'actions' in msg:
                info['has_actions'] = True
    
    info['unique_users'] = len(info['unique_users'])
    return info

# Sample 1000 threads for analysis (to speed up)
sample_size = min(1000, len(df_blueprint))
sample_threads = df_blueprint['thread'].iloc[:sample_size]

print(f"\nAnalyzing {sample_size} threads...")
thread_info = [extract_message_info(t) for t in sample_threads]
thread_info = [t for t in thread_info if t is not None]

if thread_info:
    avg_messages = np.mean([t['num_messages'] for t in thread_info])
    avg_users = np.mean([t['unique_users'] for t in thread_info])
    print(f"Average messages per thread: {avg_messages:.2f}")
    print(f"Average unique users per thread: {avg_users:.2f}")

## 2. Reddit Moderator Perceptions Dataset

In [ ]:
# Load Reddit Moderator Perceptions dataset
# Update with actual loading method based on dataset availability
# Example: df_reddit = pd.read_csv('path_to_reddit_dataset.csv')
# Or from GitHub: df_reddit = pd.read_json('https://...')

# Placeholder - update with actual source
print("Load Reddit Moderator Perceptions dataset here")
# df_reddit.head()

In [ ]:
# Basic statistics for Reddit dataset
# print(f"Shape: {df_reddit.shape}")
# print(f"\nColumns: {df_reddit.columns.tolist()}")
# print(f"\nMissing values:")
# print(df_reddit.isnull().sum())

In [ ]:
# Explore moderator perception categories
# Analyze toxicity labels, enforcement actions, etc.
# for col in categorical_cols:
#     print(f"\n{col} distribution:")
#     print(df_reddit[col].value_counts())

## 3. Summary and Next Steps

In [ ]:
# Summary of key findings
print("="*60)
print("KEY FINDINGS - BluePrint Dataset")
print("="*60)

print(f"\n1. DATASET SIZE:")
print(f"   - Total threads: {len(df_blueprint):,}")
print(f"   - Number of user clusters/personas: {df_blueprint['cluster_id'].nunique()}")

print(f"\n2. DATA STRUCTURE:")
print(f"   - Each row contains a conversation thread")
print(f"   - Threads contain sequences of messages/interactions")
print(f"   - Users grouped into {df_blueprint['cluster_id'].nunique()} behavioral personas")

print(f"\n3. RELEVANCE TO PROJECT:")
print(f"   - Can identify different user archetypes from cluster_id")
print(f"   - Thread structure allows modeling conversation dynamics")
print(f"   - Suitable for training LLM agents on realistic interactions")

print(f"\n4. NEXT STEPS:")
print(f"   - Map clusters to user archetypes (casual, influencer, toxic)")
print(f"   - Extract and preprocess message content")
print(f"   - Prepare training data for RLHF")
print(f"   - Identify or annotate toxic/moderation-relevant content")